In [ ]:
/* Add Deps */
%AddDeps edu.stanford.nlp stanford-corenlp 3.7.0
%AddDeps com.google.protobuf protobuf-java 2.6.1
%AddDeps com.databricks spark-csv_2.10 1.5.0 --transitive

// Non-repo dependencies 
%AddJar file:lib/corenlp-models.jar
%AddJar file:SE/target/scala-2.10/se_2.10-1.1.jar

In [ ]:
import com.evan.kaggle.se.FeatureEngineering._
val sqlContext = org.apache.spark.sql.SQLContext.getOrCreate(sc)
import sqlContext.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row

In [ ]:
val files = List("cooking", "crypto", "robotics", "biology", "travel", "diy")

/* Load and print all files */
val df_all = files.map(f => {
                        sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("../dat/"+f+"_clean.csv")
                   }).reduce(_ unionAll _).withColumn("tags", split($"tags", " "))

In [ ]:
df_all.show

In [ ]:
val rdd = df_all.select("title", "content").rdd.map{case Row(t: String, c: String) => (t, c)}

In [ ]:
val twoGramFeats = makeStdFeatures(2) _
val featDF = rdd.flatMap(x => twoGramFeats(x._1, x._2)).toDF

In [ ]:
featDF.show

In [ ]:
featDF.count